***Group Number 18***

Group Members

Manav Jain - 2018A7PS0102G

Rohan Agarwal - 2018A7PS0123G

Mane Digvijaysinh Sujaysinh - 2018A7PS0093G

Vishisht Agarwal - 2018A7PS0224G

In [ ]:
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
document=open("/content/drive/My Drive/IR_assign/documents.txt", 'r')
relevance_assessment=open("/content/drive/My Drive/IR_assign/relevance_assessment.txt", 'r')

## **TASK 1**

PARSING DOCUMENTS FROM QUERIES


In [ ]:
docs=document.read()
relevant = relevance_assessment.read()

In [ ]:
docs

In [ ]:
p=docs.split('/')
p = [x.replace("\n"," ") for x in p]
p=[x.strip() for x in p]
p.pop()

In [ ]:
p

In [ ]:
type(relevant)

In [ ]:
r=relevant.split('/')
r = [x.replace("\n"," ") for x in r]
r=[x.strip() for x in r]
r.pop()

In [ ]:
r

In [ ]:
relevant_dict = {}
relevant_docs = []
for s in r:
  t1 = s.split() 
  t2 = t1[1:]
  relevant_docs.append(t2) 
  for i in t2:
    relevant_dict[i]=1


In [ ]:
relevant_dict

In [ ]:
relevant_docs

In [ ]:
documents={}
for s in p:
  t1 = s.split() 
  t2 = t1[1:] 
  s2 = ' '.join(t2)
  if (t1[0] in relevant_dict.keys()):
      documents[int(t1[0])] = s2

In [ ]:
documents

INVERTED INDEX


In [ ]:
class getInvertedIndex(object):
  def __init__(self,docs):
    self.docs = docs
    self.termList = []
    self.docLists = []
    self.final_dict = {}

    for index in docs.keys():
      for term in docs[index].split(" "):
        if term in self.termList:
          i=self.termList.index(term)
          if (index) not in self.docLists[i]:
            self.docLists[i].append(index)
        else:
          self.termList.append(term)
          self.docLists.append([index])
    inverted_dict={}
    for terms in range(len(self.termList)):
      inverted_dict[self.termList[terms]] = self.docLists[terms]
    dict_len= {key: len(value) for key, value in inverted_dict.items()}
    import operator
    sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
    sorted_dict = [{item[0]: inverted_dict[item [0]]} for item in sorted_key_list]
    for i in range(len(sorted_dict)):
      for j in sorted_dict[i]:
        self.final_dict[j] = sorted_dict[i][j]
     
  def search(self,term):
    try:
      i=self.termList.index(term)
      return self.docLists[i]
    except:
      return "No results"

In [ ]:
myInvertedIndex=getInvertedIndex(documents)

In [ ]:
myInvertedIndex.final_dict

VOCAB SIZE

In [ ]:
len (myInvertedIndex.termList)

PARSING QUERY

In [ ]:
queryDoc=open("/content/drive/My Drive/IR_assign/query.txt", 'r')

In [ ]:
queryInit=queryDoc.read()

In [ ]:
queryFin=queryInit.split('/')
queryFin = [x.replace("\n"," ") for x in queryFin]
queryFin=[x.strip() for x in queryFin]

In [ ]:
originalQueries={}
for i in range(len(queryFin)-1):
  s = queryFin[i]
  t1 = s.split()
  t2 = t1[1:] 
  s2 = ' '.join(t2)
  s2 = s2.lower();
  t2 = s2.split();
  originalQueries[int(t1[0])]=[]
  for idx in range(len(t2)):
    originalQueries[int(t1[0])].append(t2[idx])

In [ ]:
type(originalQueries)

In [ ]:
queries={}
for i in range(1,11):
  queries[i]=[]
  for word in originalQueries[i]:
      queries[i].append(word)

In [ ]:
queries

PROCESSING 'OR' QUERY


In [ ]:
def orQuery(list1, list2):
  ans=[]
  p1=0
  p2=0
  while (p1<len(list1) and p2<len(list2)):
    if (list1[p1] == list2[p2]):
      ans.append(list1[p1])
      p1=p1+1
      p2=p2+1
    else:
      if list1[p1] > list2[p2]:
        ans.append(list2[p2])
        p2=p2+1
      else:
        ans.append(list1[p1])
        p1=p1+1
  while p1<len(list1):
    ans.append(list1[p1])
    p1=p1+1
  while p2<len(list2):
    ans.append(list2[p2])
    p2=p2+1
  return ans

RETRIEVING DOCUMENTS WITHOUT ANY LINGUISTIC MODEL

In [ ]:
docsRetrieved={}
for q in range(1,11):
  docsRetrieved[q]=[]
  for word in queries[q]:
    if word in myInvertedIndex.final_dict:
      docsRetrieved[q] = orQuery(docsRetrieved[q], myInvertedIndex.final_dict[word])

In [ ]:
def andQuery(list1, list2):
  ans=0
  p1=0
  p2=0
  while (p1<len(list1) and p2<len(list2)):
    if (list1[p1] == int(list2[p2])):
      ans=ans+1
      p1=p1+1
      p2=p2+1
    else:
      if list1[p1] > int(list2[p2]):
        p2=p2+1
      else:
        p1=p1+1
  return ans

In [ ]:
relevant_docs

PRECISION & RECALL

In [ ]:
precision=[]
recall=[]
for q in range(1,11):
  relevantRetrieved=andQuery(docsRetrieved[q], relevant_docs[q-1])
  precision.append(relevantRetrieved/len(docsRetrieved[q]))
  recall.append(relevantRetrieved/len(relevant_docs[q-1]))

In [ ]:
import matplotlib.pyplot as plt 
queryID = [i+1 for i in range(len(precision))]
plt.plot(queryID,precision,label = "precision")
plt.plot(queryID,recall,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

## **TASK 2**

LINGUISTIC MODELS

In [ ]:
import nltk

In [ ]:
nltk.download()

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
spacy_processed_docs={}
for d in documents.keys():
  spacy_processed_docs[d] = (nlp(documents[d]))

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS)
print(type(STOP_WORDS))

USING SPACY STOP WORDS TO IMPROVE PRECISION

In [ ]:
post_stop_word_removal={}
for i in spacy_processed_docs.keys():
  post_stop_word_removal[i] = [token.text for token in spacy_processed_docs[i] if not token.is_stop]

In [ ]:
spacyDocs={}
seperator=' '
for i in spacy_processed_docs.keys():
  spacyDocs[i] = seperator.join(post_stop_word_removal[i])

In [ ]:
myInvertedIndexP=getInvertedIndex(spacyDocs)

In [ ]:
spacy_processed_query=[]
for i in range(1,11):
  temp=seperator.join(originalQueries[i])
  spacy_processed_query.append(nlp(temp))

In [ ]:
originalQueries

In [ ]:
spacy_processed_query

In [ ]:
post_removal={}
for i in range (10):
  post_removal[i] = [token.text for token in spacy_processed_query[i] if not token.is_stop]

In [ ]:
post_removal

RETRIEVING DOCS AFTER REMOVING SPACY STOP WORDS

In [ ]:
spacy_docsRetrieved={}
for q in range(10):
  spacy_docsRetrieved[q]=[]
  for word in post_removal[q]:
    if word in myInvertedIndexP.final_dict:
      spacy_docsRetrieved[q] = orQuery(spacy_docsRetrieved[q], myInvertedIndexP.final_dict[word])

PRECISION AND RECALL

In [ ]:
spacy_precision=[]
spacy_recall=[]
for q in range(10):
  relevantRetrieved=andQuery(spacy_docsRetrieved[q], relevant_docs[q])
  spacy_precision.append(relevantRetrieved/len(spacy_docsRetrieved[q]))
  spacy_recall.append(relevantRetrieved/len(relevant_docs[q]))

In [ ]:
spacy_queryID = [i+1 for i in range(len(spacy_precision))]
plt.plot(spacy_queryID,spacy_precision,label = "precision")
plt.plot(spacy_queryID,spacy_recall,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

USING LANCASTER STEMMER TO IMPROVE RECALL

In [ ]:
from nltk.stem.snowball import PorterStemmer,SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer

In [ ]:
ls = LancasterStemmer()

In [ ]:
stemmer_processed_docs={}
for d in documents.keys():
  li = list(documents[d].split(" ")) 
  for i in range(len(li)):
    li[i]=ls.stem(li[i])
  stemmer_processed_docs[d] = seperator.join(li)

In [ ]:
myInvertedIndexR=getInvertedIndex(stemmer_processed_docs)

In [ ]:
stemmer_processed_query=[]
for i in range(1,11):
  li=[]
  for j in range(len(originalQueries[i])):
    li.append(ls.stem(originalQueries[i][j]))
  stemmer_processed_query.append(seperator.join(li))

RETRIEVING DOCS AFTER USING LANCASTER STEMMER

In [ ]:
stemmer_docsRetrieved={}
for q in range(10):
  stemmer_docsRetrieved[q]=[]
  for word in (stemmer_processed_query[q].split(" ")):
    if word in myInvertedIndexR.final_dict:
      stemmer_docsRetrieved[q] = orQuery(stemmer_docsRetrieved[q], myInvertedIndexR.final_dict[word])

PRECISION AND RECALL

In [ ]:
stemmer_precision=[]
stemmer_recall=[]
for q in range(10):
  relevantRetrieved=andQuery(stemmer_docsRetrieved[q], relevant_docs[q])
  stemmer_precision.append(relevantRetrieved/len(stemmer_docsRetrieved[q]))
  stemmer_recall.append(relevantRetrieved/len(relevant_docs[q]))

In [ ]:
stemmer_queryID = [i+1 for i in range(len(stemmer_precision))]
plt.plot(stemmer_queryID,stemmer_precision,label = "precision")
plt.plot(stemmer_queryID,stemmer_recall,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

##**TASK 3**

***Task3 A:***
Changes made in the dataset for improving on precision:
Removal of stop-words using spacy:
We observed that there were instances of documents having a lot of stop words like “as”, “in”  etc. Owing to these, a lot of non-relevant documents were getting retrieved.
	
Changes made in the dataset for improving on recall:
We did try to remove stop words, using spacy, but, unexpectedly this led to a decrease in recall. 
*For Ex*. One document had a lot of stop words like “is”, “an” etc. but also had a relevant word *“mechanisms”*. Even though my query had *“mechanism”* but the reason it was initially retrieved was because of overlapping stop words. Now that all the stop words were removed and no other linguistic model was used, this document did not get retrieved.
Use of stemming:
We decided to go with Porter’s stemmer but we observed that only the following conversions were being made equivalent: 
Controlled --- control
We tried LancasterStemmer after that and in addition to the making the above two words equivalent, it was also making the following conversions:
Amplifiers --- amplification

***Task3 B:***
Changes in Recall for respective queries:
Before using stemming :
[1.00, 1.00, 1.00, 1.00, 1.00, 0.96, 0.91, 0.87, 1.00, 1.00]
After using stemming:
[1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.95, 0.87, 1.00, 1.00]                              

 Changes in Precision for respective queries:
 Before stopword removal:
[0.15, 0.17, 0.07, 0.02, 0.13, 0.30, 0.13, 0.11, 0.24, 0.04]
After stopword removal:
[0.36, 0.36, 0.50, 0.12, 0.37, 0.30, 0.35, 0.12, 0.24, 0.22]


## **TASK 4**

In [ ]:
def and_query(list1, list2):
  ans=[]
  p1=0
  p2=0
  while (p1<len(list1) and p2<len(list2)):
    if (list1[p1] == list2[p2]):
      ans.append(list1[p1])
      p1=p1+1
      p2=p2+1
    else:
      if list1[p1] > list2[p2]:
        p2=p2+1
      else:
        p1=p1+1
  return ans

In [ ]:
bi_gram_dict={}
for word in myInvertedIndex.final_dict.keys():
  temp = '$' + word + '$'
  for i in range(len(temp)-1):
    bi = temp[i]+temp[i+1]
    if (bi in bi_gram_dict.keys()):
      if (word not in bi_gram_dict[bi]):
        bi_gram_dict[bi].append(word)
    else:
      bi_gram_dict[bi] = [word]

In [ ]:
for bi in bi_gram_dict.keys():
  bi_gram_dict[bi].sort()

In [ ]:
for q in queries.keys():
  queries[q][2] = queries[q][2][:2] + '*' + queries[q][2][2:]
  queries[q][4] = '*' + queries[q][4]
  queries[q][1] = queries[q][1] + '*'
  


In [ ]:
doc_ret = {}
for q in queries.keys():
  doc_ret[q] = []
  for word in queries[q]:
    if ('*' in word):
      temp = '$'+word+'$'
      
      word_list = []
      f=0
      for i in range(len(temp)-1):
        if (temp[i]!='*' and temp[i+1]!='*'):
          bi = temp[i]+temp[i+1]
          if (f==0):
            word_list = bi_gram_dict[bi]
            f=1
          else:
            word_list = and_query(word_list, bi_gram_dict[bi])
      for w in word_list:
        if (w in myInvertedIndex.final_dict.keys()):
          doc_ret[q] = orQuery(doc_ret[q], myInvertedIndex.final_dict[w])

    else:
      if (word in myInvertedIndex.final_dict.keys()):
        doc_ret[q] = orQuery(doc_ret[q], myInvertedIndex.final_dict[word])

PRECISION AND RECALL WITHOUT ANY LINGUISTIC MODEL

In [ ]:
bi_precision=[]
bi_recall=[]
for q in range(1,11):
  relevantRetrieved=andQuery(doc_ret[q], relevant_docs[q-1])
  bi_precision.append(relevantRetrieved/len(doc_ret[q]))
  bi_recall.append(relevantRetrieved/len(relevant_docs[q-1]))

print(bi_precision)
print(bi_recall)

In [ ]:
plt.plot(queryID,bi_precision,label = "precision")
plt.plot(queryID,bi_recall,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

In [ ]:
bi_gram_dict_P={}
for word in myInvertedIndexP.final_dict.keys():
  temp = '$' + word + '$'
  for i in range(len(temp)-1):
    bi = temp[i]+temp[i+1]
    if (bi in bi_gram_dict_P.keys()):
      if (word not in bi_gram_dict_P[bi]):
        bi_gram_dict_P[bi].append(word)
    else:
      bi_gram_dict_P[bi] = [word]

In [ ]:
for bi in bi_gram_dict_P.keys():
  bi_gram_dict_P[bi].sort()

In [ ]:
for q in post_removal.keys():
  post_removal[q][2] = post_removal[q][2][:2] + '*' + post_removal[q][2][2:]
  post_removal[q][4] = '*' + post_removal[q][4]
  post_removal[q][1] = post_removal[q][1] + '*'

In [ ]:
doc_ret_P = {}
for q in post_removal.keys():
  doc_ret_P[q] = []
  for word in post_removal[q]:
    if ('*' in word):
      temp = '$'+word+'$'
      
      word_list = []
      f=0
      for i in range(len(temp)-1):
        if (temp[i]!='*' and temp[i+1]!='*'):
          bi = temp[i]+temp[i+1]
          if (f==0):
            word_list = bi_gram_dict_P[bi]
            f=1
          else:
            word_list = and_query(word_list, bi_gram_dict_P[bi])
  
      for w in word_list:
        if (w in myInvertedIndexP.final_dict.keys()):
          doc_ret_P[q] = orQuery(doc_ret_P[q], myInvertedIndexP.final_dict[w])

    else:
      if (word in myInvertedIndexP.final_dict.keys()):
        doc_ret_P[q] = orQuery(doc_ret_P[q], myInvertedIndexP.final_dict[word])



PRECISION AND RECALL USING SPACY STOPWORDS REMOVAL

In [ ]:
bi_precision_P=[]
bi_recall_P=[]
for q in range(10):
  relevantRetrieved=andQuery(doc_ret_P[q], relevant_docs[q])
  bi_precision_P.append(relevantRetrieved/len(doc_ret_P[q]))
  bi_recall_P.append(relevantRetrieved/len(relevant_docs[q]))

print(bi_precision_P)
print(bi_recall_P)

In [ ]:
plt.plot(queryID,bi_precision_P,label = "precision")
plt.plot(queryID,bi_recall_P,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

In [ ]:
stemmer_query = {}
for i in range(10):
  stemmer_query[i] = stemmer_processed_query[i].split(' ')

In [ ]:
for q in stemmer_query.keys():
  stemmer_query[q][2] = stemmer_query[q][2][:2] + '*' + stemmer_query[q][2][2:]
  stemmer_query[q][4] = '*' + stemmer_query[q][4]
  stemmer_query[q][1] = stemmer_query[q][1] + '*'

In [ ]:
bi_gram_dict_R={}
for word in myInvertedIndexR.final_dict.keys():
  temp = '$' + word + '$'
  for i in range(len(temp)-1):
    bi = temp[i]+temp[i+1]
    if (bi in bi_gram_dict_R.keys()):
      if (word not in bi_gram_dict_R[bi]):
        bi_gram_dict_R[bi].append(word)
    else:
      bi_gram_dict_R[bi] = [word]

In [ ]:
for bi in bi_gram_dict_R.keys():
  bi_gram_dict_R[bi].sort()

In [ ]:
doc_ret_R = {}
for q in stemmer_query.keys():
  doc_ret_R[q] = []
  for word in stemmer_query[q]:
    if ('*' in word):
      temp = '$'+word+'$'
      
      word_list = []
      f=0
      for i in range(len(temp)-1):
        if (temp[i]!='*' and temp[i+1]!='*'):
          bi = temp[i]+temp[i+1]
          if (f==0):
            if (bi in bi_gram_dict_R.keys()):
              word_list = bi_gram_dict_R[bi]
              f=1
          else:
            if (bi in bi_gram_dict_R.keys()):
              word_list = and_query(word_list, bi_gram_dict_R[bi])
      if (q==1):
        print(temp,word_list)
      for w in word_list:
        if (w in myInvertedIndexR.final_dict.keys()):
          doc_ret_R[q] = orQuery(doc_ret_R[q], myInvertedIndexR.final_dict[w])

    else:
      if (word in myInvertedIndexR.final_dict.keys()):
        doc_ret_R[q] = orQuery(doc_ret_R[q], myInvertedIndexR.final_dict[word])



PRECISION AND REMOVAL USING LANCASTER STEMMER

In [ ]:
bi_precision_R=[]
bi_recall_R=[]
for q in range(10):
  relevantRetrieved=andQuery(doc_ret_R[q], relevant_docs[q])
  bi_precision_R.append(relevantRetrieved/len(doc_ret_R[q]))
  bi_recall_R.append(relevantRetrieved/len(relevant_docs[q]))

print(bi_precision_R)
print(bi_recall_R)

In [ ]:
plt.plot(queryID,bi_precision_R,label = "precision")
plt.plot(queryID,bi_recall_R,label = "recall")
plt.xlabel('QueryID') 
plt.ylabel('Performance scores') 
plt.title('Performance Graph')
plt.legend() 
plt.show() 

JUSTIFICATION :- Different wild card queries were made for all the different models and processed with the help of bi-gram indexing. The results of Precision and Recall of different models were consistent with the previous results described above. 

The precision of the full model with wildcard queries was increased with the help of using spacy stop word removal on the wildcard queries, while the recall was improved by employing Lancaster stemmer for wildcard queries.  


ASSIGNMENT-2 starts from here!!!!

all variables used in the second assignment must end with 2.

In [ ]:
myInvertedIndex

In [ ]:
originalQueries

In [ ]:
import nltk
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [ ]:
def removeStopWords(tokenizedSent):
  '''
  :params:
  tokenizedSent: list of individual tokens

  :returns:
  list of individual tokens w/o the stop words
  '''
  sw = set(stopwords.words('english'))
  return [w for w in tokenizedSent if w not in sw]

In [ ]:
post_stop_removal2={}
for i in range (1,11):
  post_stop_removal2[i] = removeStopWords(originalQueries[i])

In [ ]:
post_stop_removal2

In [ ]:
for i in range(1,11):
  for s in post_stop_removal2[i]:
    s=s.lower()

In [ ]:
post_stop_removal2

In [ ]:
type(documents[129])

In [ ]:
## HELPER FUNCTIONS

def lowerSent(sentence):
  '''
  :params:
  sentence: complete input string

  :returns:
  lower-cased input string
  '''
  return sentence.lower()

def tokenizeSent(sentence):
  '''
  :params:
  sentence: complete input string

  :returns:
  list of individual tokens
  '''
  tokenizer = RegexpTokenizer("[a-zA-Z@]+")
  return tokenizer.tokenize(sentence)

def removeStopWords(tokenizedSent):
  '''
  :params:
  tokenizedSent: list of individual tokens

  :returns:
  list of individual tokens w/o the stop words
  '''
  sw = set(stopwords.words('english'))
  return [w for w in tokenizedSent if w not in sw]
  
def stem(tokenizedSent):
  '''
  :params:
  tokenizedSent: list of individual tokens

  :returns:
  list of stemmed individual tokens
  '''
  stemmer = PorterStemmer()
  return [stemmer.stem(w) for w in tokenizedSent]

In [ ]:
def myTokenizer(sentence):
  '''
  :params:
  sentence: complete input string

  :returns:
  list of preprocessed individual tokens
  '''
  loweredSent = lowerSent(sentence)
  tokenizedSent = tokenizeSent(loweredSent)
  tokenizedSent = removeStopWords(tokenizedSent)
  stemmedSent = stem(tokenizedSent)
  preprocessedSent = stemmedSent

  return preprocessedSent

In [ ]:
def getTokens(sentence):
  tokens=[]
  t1 = sentence.split() 
  t2 = t1[1:]

  return t2


In [ ]:
from collections import defaultdict,Counter

In [ ]:
def get_tf_doc(doc_id):
    """
    :params:
    docId: Id of a document

    :returns:
    a dictionary correpsonding to token:frequency(token) of tokens present in the document represented by docId 
  
"""
    tokens=getTokens(documents[doc_id])
    tf_dict = Counter(tokens)
    return tf_dict

In [ ]:
get_tf_doc(129)

In [ ]:
doc_ids=[]
for ids in documents.keys():
  doc_ids.append(ids)

In [ ]:
docs_tf_dict = {}
for docId in doc_ids:
    docs_tf_dict[docId] = get_tf_doc(docId)

In [ ]:
docs_tf_dict

In [ ]:
def read_doc(docId):
    """
    Utility function to read the contenst of a document given its docId
    """
    with open(os.path.join(os.getcwd(),"Working_Sample",docId),'r') as f:
        doc_text=f.read()
    return doc_text

In [ ]:
def get_df(doc_ids):
    '''
    :params:
    doc_ids: set of documents [corpus]

    :returns:
     a dictionary corresponding to token:Document Frequency(token) i.e the number of documents that token is present in
    '''

    result_dict = None
    for doc in doc_ids:
        doc_tokens = getTokens(documents[doc])
        temp_df_dict = Counter(set(doc_tokens))
        if result_dict:
            result_dict = result_dict + temp_df_dict
        else:
            result_dict = temp_df_dict
    return result_dict

In [ ]:
terms_df_dict=get_df(doc_ids)

In [ ]:
from math import log10

In [ ]:
idf_dict = {}
for key,val in terms_df_dict.items():
    idf_dict[key] = log10(len(documents)/float(val)) 
    ## AT TIMES N/(1+ df) IS USED TO HANDLE THE CASE WHERE DF = 0, HOWEVER, THAT CAN LEAD TO IDF BEING NEGATIVE
print("The idf dictionary is:")
idf_dict

In [ ]:
def TFIDFRetrieval(query, documents, docs_tf_dict=None, idf_dict=None):
  '''
  :params:
  query: user input [query string]
  documents: set of documents [corpus]
  docs_tf_dict: dictionary of term-frequencies for each document [OPTIONAL]
  idf_dict: dictionary of inverse document frequencies for each term in corpus [OPTIONAL]

  :returns:
  list of sorted [decreasing] tuples where each tuple comprises of the document ID and the associated score
  '''
  if docs_tf_dict == None:
    docs_tf_dict = {}
    for docId in documents:
      docs_tf_dict[docId] = get_tf_doc(docId)
  
  if idf_dict == None:
    df_dict = get_df(documents)
    idf_dict = {}
    for key,val in df_dict.items():
      idf_dict[key] = log10(len(spacyDocs)/float(val))

  query_tokens = query
  results = []
  for doc in documents:
    score = 0
    for token in query_tokens:
      try:
        score += (log10(docs_tf_dict[doc][token]+1) * idf_dict[token])
      except:
        score += 0
    results.append([score,doc])
  return sorted(results, reverse=True)

In [ ]:
TFIDFRetrieval(post_stop_removal2[1],doc_ids, docs_tf_dict=docs_tf_dict, idf_dict=idf_dict)

In [ ]:
print("Query IDs | \tDoc IDs \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t| \tScores ")
for qid,q in originalQueries.items():
  results=TFIDFRetrieval(q,documents, docs_tf_dict=docs_tf_dict, idf_dict=idf_dict)
  print(qid, "\t  | ",end="\t")
  for i in range (20):
    print("{0:d} ".format(results[i][1]),end="\t")
  print(" \t| ", end="\t")
  for i in range (20):
    print("{0:2.2f} " .format(results[i][0]),end="\t")
  print()

Top K precision without any modification

In [ ]:
r[9]

In [ ]:
rel_docs_dict={}
for str in r:
  t1=str.split(' ');
  t2=t1[1:]
  q_no=int(t1[0])
  for s in t2:
    if s=='':
      continue
    s=int(s)
    if s not in rel_docs_dict:
      rel_docs_dict[s]=[]
    rel_docs_dict[s].append(q_no)

In [ ]:
rel_docs_dict

In [ ]:
precision_k={}
total=0
rel=0
for qid,q in post_stop_removal2.items():
  total=0
  rel=0
  results=TFIDFRetrieval(q,documents, docs_tf_dict=docs_tf_dict, idf_dict=idf_dict)
  print(qid, "\t  | ",end="\t")
  precision_k[qid]=[]
  for i in range (20):
    total+=1
    if results[i][1] in rel_docs_dict:
      for s in rel_docs_dict[results[i][1]]:
        if(s==qid):
          rel+=1
          break
    precision_k[qid].append(rel/total)
    print("{0:d} ".format(results[i][1]),end="\t")
  print(" \t| ", end="\t")
  for i in range (20):
    print("{0:2.2f} " .format(results[i][0]),end="\t")
  print()

In [ ]:
precision_k

**Increase** **Precision**

In [ ]:
spacyDocs

In [ ]:
def get_tf_doc_spacy(doc_id):
    """
    :params:
    docId: Id of a document

    :returns:
    a dictionary correpsonding to token:frequency(token) of tokens present in the document represented by docId 
  
"""

    tokens=myTokenizer(spacyDocs[doc_id])
    print(tokens)
    tf_dict = Counter(tokens)
    return tf_dict

In [ ]:
spacy_tf_dict={}
for docId in doc_ids:
    spacy_tf_dict[docId] = get_tf_doc_spacy(docId)

In [ ]:
spacy_tf_dict

In [ ]:
def get_df_spacy(doc_ids):
    '''
    :params:
    doc_ids: set of documents [corpus]

    :returns:
     a dictionary corresponding to token:Document Frequency(token) i.e the number of documents that token is present in
    '''

    result_dict = None
    for doc in doc_ids:
        doc_tokens = myTokenizer(spacyDocs[doc])
        temp_df_dict = Counter(set(doc_tokens))
        if result_dict:
            result_dict = result_dict + temp_df_dict
        else:
            result_dict = temp_df_dict
    return result_dict

In [ ]:
import copy
spacy_df_dict = copy.deepcopy(get_df_spacy(doc_ids))


In [ ]:
spacy_df_dict

In [ ]:
spacy_idf_dict = {}
for key,val in spacy_df_dict.items():
    spacy_idf_dict[key] = log10(len(spacyDocs)/float(val)) 
    ## AT TIMES N/(1+ df) IS USED TO HANDLE THE CASE WHERE DF = 0, HOWEVER, THAT CAN LEAD TO IDF BEING NEGATIVE
print("The idf dictionary is:")
spacy_idf_dict

In [ ]:
TFIDFRetrieval(post_stop_removal2[1], doc_ids, docs_tf_dict=spacy_tf_dict, idf_dict=spacy_idf_dict)

In [ ]:
print("Query IDs | \tDoc IDs \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t| \tScores ")
for qid,q in post_stop_removal2.items():
  results=TFIDFRetrieval(q,spacyDocs, docs_tf_dict=spacy_tf_dict, idf_dict=spacy_idf_dict)
  print(qid, "\t  | ",end="\t")
  for i in range (20):
    print("{0:d} ".format(results[i][1]),end="\t")
  print(" \t| ", end="\t")
  for i in range (20):
    print("{0:2.2f} " .format(results[i][0]),end="\t")
  print()

In [ ]:
spacy_precision_k={}
total=0
rel=0
for qid,q in post_stop_removal2.items():
  total=0
  rel=0
  results=TFIDFRetrieval(q,spacyDocs, docs_tf_dict=spacy_tf_dict, idf_dict=spacy_idf_dict)
  print(qid, "\t  | ",end="\t")
  spacy_precision_k[qid]=[]
  for i in range (20):
    total+=1
    if results[i][1] in rel_docs_dict:
      for s in rel_docs_dict[results[i][1]]:
        if(s==qid):
          rel+=1
          break
    spacy_precision_k[qid].append(rel/total)
    print("{0:d} ".format(results[i][1]),end="\t")
  print(" \t| ", end="\t")
  for i in range (20):
    print("{0:2.2f} " .format(results[i][0]),end="\t")
  print()

In [ ]:
spacy_precision_k

In [ ]:
post_stop_removal2[3]

In [ ]:
print(originalQueries)


## Task2

In [ ]:
import copy
spell_error_queries = copy.deepcopy(post_stop_removal2)
print(spell_error_queries)

In [ ]:
import random
import string
for query_id in spell_error_queries.keys():
  temp = []
  for i in range(len(spell_error_queries[query_id] )):
    if len(spell_error_queries[query_id][i])>2:
      temp.append(i)
  r1 = random.choice(temp)
  w1 = spell_error_queries[query_id][r1]
  delchar = random.randint(1,len(w1)-1)
  tw1 = w1[:delchar-1]+ w1[delchar:]
  spell_error_queries[query_id][r1] = tw1
  temp.remove(r1)

  r2 = random.choice(temp)
  w2 = spell_error_queries[query_id][r2]
  delchar = random.randint(1,len(w2)-1)
  tw2 = w2[:delchar-1]+ w2[delchar:]
  r22 = random.randint(0,len(tw2)-1)
  tw22 = tw2[:r22] + random.choice(string.ascii_lowercase) + tw2[r22:]
  spell_error_queries[query_id][r2] = tw22
  temp.remove(r2)

  r3 = random.choice(temp)
  w3 = spell_error_queries[query_id][r3]
  l = len(w3)
  pos1 = random.randint(0,l//2-1)
  pos2 = random.randint(l//2+1,l-1)
  w33 = [char for char in  w3]
  c = w33[pos1]
  w33[pos1]= w33[pos2]
  w33[pos2] = c
  fw3 = ''.join(w33)
  spell_error_queries[query_id][r3] = fw3
  print(w1,w2,w3)

In [ ]:
def editDistDP(str1, str2):
    m = len(str1);
    n = len(str2); 
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0: 
                dp[i][j] = j 
            elif j == 0: 
                dp[i][j] = i 
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
            else: 
                dp[i][j] = min(1+dp[i][j-1], 1+dp[i-1][j], 2+dp[i-1][j-1])
  
    return dp[m][n] 

In [ ]:
myInvertedIndex2 = getInvertedIndex(documents)

In [ ]:
corrected_queries = {}
from prettytable import PrettyTable
t = PrettyTable(['Query IDs', 'Typo', 'Term', 'Distance'])
for i in spell_error_queries.keys():
  corrected_queries[i] = []
  for word in spell_error_queries[i]:
    mini = 5
    corr_word = word
    listRow = [];
    listRow.append(i);
    listRow.append(word);
    for original_word in myInvertedIndex2.final_dict.keys():
      if original_word in STOP_WORDS:
        continue
      if (editDistDP(word, original_word)<mini):
        mini = editDistDP(word, original_word)
        corr_word = original_word
    listRow.append(corr_word);
    listRow.append(mini);
    corrected_queries[i].append(corr_word)
    if corr_word != word:
      t.add_row(listRow);
print(t);

In [ ]:
print(spell_error_queries)

In [ ]:
print(corrected_queries[1])

In [ ]:
precision_k_spell={}
total=0
rel=0
for qid,q in corrected_queries.items():
  total=0
  rel=0
  results=TFIDFRetrieval(q,documents)
  precision_k_spell[qid]=[]
  for i in range(5):
    total+=1
    if results[i][1] in rel_docs_dict:
      for s in rel_docs_dict[results[i][1]]:
        if(s==qid):
          rel+=1
          break
    precision_k_spell[qid].append(rel/total)
    print("{0:d} ".format(results[i][1]),end="\t")
  for i in range (5):
    print("{0:2.2f} " .format(results[i][0]),end="\t")
  print()
precision_k_spell

In [ ]:
#fingerkeying - an implementation of QWERTY Keyboard
#Author Ekta Grover ekta1007@gmail.com on 19th July, 2013

from __future__ import division
from itertools import chain
#import pydot
from random import random
import matplotlib.pyplot as plt
import networkx as nx


G=nx.Graph()
# Keying in the structure of the QWERTY keyboard
row1=['q','w','e','r','t','y','u','i','o','p']
row2=['a','s','d','f','g','h','j','k','l']
row3=['z','x','c','v','b','n','m']

# Dividing the keyboard in left and right 

right_keys=list(chain.from_iterable([row1[0:5],row2[0:5],row3[0:5]]))
left_keys=list(chain.from_iterable([row1[5:10],row2[5:9],row3[5:7]]))

# add each rows independetly with the corresponding weights - we will use only weight of 1 , so we connect only adjacent nodes
for i in range(0,len(row1)-1):
    G.add_edge(row1[i],row1[i+1],weight=1,color='red')
  
for i in range(0,len(row2)-1):
    G.add_edge(row2[i],row2[i+1],weight=1,color='blue')
    
for i in range(0,len(row3)-1):
    G.add_edge(row3[i],row3[i+1],weight=1,color='green')

# add the next row in there - row1 with row2
for i in range(0, len(row1)):
    if i==0:
        G.add_edge(row1[i],row2[i],weight=1,color='cyan')
        G.add_edge(row1[i],row2[i+1],weight=1,color='cyan')
    elif i<= 7 :
        G.add_edge(row1[i],row2[i-1],weight=1,color='cyan')
        G.add_edge(row1[i],row2[i],weight=1,color='cyan')
        G.add_edge(row1[i],row2[i+1],weight=1,color='cyan')
    elif i==8:
        G.add_edge(row1[i],row2[i-1],weight=1,color='cyan')
        G.add_edge(row1[i],row2[i],weight=1,color='cyan')
    elif i==9:
        G.add_edge(row1[i],row2[i-1],weight=1,color='cyan')

# add the next row in there -- row2 with row3
for i in range(0, len(row2)):
    if i==0:
        G.add_edge(row2[i],row3[i],weight=1,color='black')
        G.add_edge(row2[i],row3[i+1],weight=1,color='black')
    elif i<= 5 :
        G.add_edge(row2[i],row3[i-1],weight=1,color='black')
        G.add_edge(row2[i],row3[i],weight=1,color='black')
        G.add_edge(row2[i],row3[i+1],weight=1,color='black')
    elif i==6:
        G.add_edge(row2[i],row3[i-1],weight=1,color='black')
        G.add_edge(row2[i],row3[i],weight=1,color='black')
    elif i==7:
        G.add_edge(row2[i],row3[i-2],weight=1,color='black')
        G.add_edge(row2[i],row3[i-1],weight=1,color='black')
    elif i==8:
        G.add_edge(row2[i],row3[i-2],weight=1,color='black')

# added all unidirected graphs with weights   

"""
#Good to see & possibly debug the structure of the graph as follows -
G.number_of_nodes()
G.number_of_edges()
list_of_all_nodes=[row1,row2,row3]
#flatten all_nodes as a single list - of which we can find the length, and see all edges together
list_of_all_nodes=list(chain.from_iterable(all_nodes))
"""

edgelist1=[(u,v) for (u,v,y) in G.edges(data=True) if y['color']=='red']
edgelist2=[(u,v) for (u,v,y) in G.edges(data=True) if y['color']=='blue']
edgelist3=[(u,v) for (u,v,y) in G.edges(data=True) if y['color']=='green']
edgelist4=[(u,v) for (u,v,y) in G.edges(data=True) if y['color']=='black']
edgelist5=[(u,v) for (u,v,y) in G.edges(data=True) if y['color']=='cyan']

pos=nx.spring_layout(G)
#color coding the rows of the keyboard differently, and picking up the labels of the nodes
nx.draw_networkx_nodes(G,pos,nodelist=row1,node_color='red',node_size=400)
nx.draw_networkx_nodes(G,pos,nodelist=row2,node_color='blue',node_size=400)
nx.draw_networkx_nodes(G,pos,nodelist=row3,node_color='green',node_size=400)

nx.draw_networkx_edges(G,pos,edgelist1,width=4,edge_color='red')
nx.draw_networkx_edges(G,pos,edgelist2,width=4,edge_color='blue')
nx.draw_networkx_edges(G,pos,edgelist3,width=4,edge_color='green')
nx.draw_networkx_edges(G,pos,edgelist4,width=3,edge_color='black')
nx.draw_networkx_edges(G,pos,edgelist5,width=3,edge_color='cyan')

# labels
nx.draw_networkx_labels(G,pos,font_size=15,font_family='calibri')

plt.axis('off')
plt.savefig("finger_keying.png")
#plt.show() # display the QWERTY represntation graph 

"""
Base assumptions for the algorithm 
1. Two words have more probabilty to be similar if the difference of their lengths does not exceed a threshold - we take this threshold as 1/3rd of the minimum of the two words keyed in
2. Will use the LevenshteinDistance if the differnce in lenghts exceeds a threshold - meaning it must not be a typo, and we must find the regular distance metric
"""

connected_node_list=G.edges()

def Qwerty_dist(word1,word2):
    dist=0
    number_passes_adjacency=0
    number_passes_direction=0
    # It can't be that most of the characters of two words are mistyped- so we prune the results, by setting another threshold for hwo many ignores are possible
    threshold=(min(len(word1),len(word2)))/3
    number_passes_direction_threshold= (min(len(word1),len(word2)))/2
    number_passes_adjacency_threshold= (min(len(word1),len(word2)))/2
    if abs(len(word1)-len(word2))<=threshold:
        for i in range(0,min(len(word1),len(word2))):
            if word1[i]==word2[i]:
                pass
            elif (word1[i],word2[i]) or (word2[i],word1[i]) in connected_node_list :
                #meaning the intention of the user must have been same and it's indeed a typo
                number_passes_adjacency=number_passes_adjacency+1
                #pass
            elif (word1[i],word2[i]) or (word2[i],word1[i]) not in connected_node_list :
            #looks like the characters in the words just got reversed, we place the restriction that the two swapped "keys" happen to come from left & right section of keyboard
                if i<=min(len(word1),len(word2))-2 :
                    if (word1[i] in left_keys and word2[i] in right_keys) or (word1[i] in right_keys and word2[i] in left_keys):
                        #either the keys are exactly reversed 
                        if word1[i]==word2[i+1] and word1[i+1]==word2[i]:
                            number_passes_direction=number_passes_direction+1
                            #pass
                        #otherwise, it could be that the "swapped" words are "neighborhood" connected, hence looking up in connected_node_list- ie we swapped, but we did a typo !
                        elif (word1[i],word2[i+1]) and (word1[i+1],word2[i]) in connected_node_list :
                            number_passes_adjacency=number_passes_adjacency+1
                            #pass
                        else :
                            dist = dist+1
            else: 
                dist=dist+1
        if dist ==0 and number_passes_direction <= number_passes_direction_threshold  and number_passes_adjacency <= number_passes_adjacency_threshold :
            return 0;
        elif dist>0 and number_passes_direction <= number_passes_direction_threshold  and number_passes_adjacency <= number_passes_adjacency_threshold :
            return dist;
        elif number_passes_direction > number_passes_direction_threshold  or number_passes_adjacency > number_passes_adjacency_threshold :
            dist=editDistDP(word1,word2)
            return dist;
    else:
    #if abs(len(word1)-len(word2))>threshold :
        dist=editDistDP(word1,word2)
        return dist;

In [ ]:
corrected_queries_qwerty = {}
from prettytable import PrettyTable
t = PrettyTable(['Query IDs', 'Typo', 'Term', 'Distance'])
for i in spell_error_queries.keys():
  corrected_queries_qwerty[i] = []
  for word in spell_error_queries[i]:
    mini = 5
    corr_word = word
    listRow = [];
    listRow.append(i);
    listRow.append(word);
    for original_word in myInvertedIndex2.final_dict.keys():
      if original_word in STOP_WORDS:
        continue
      temp = Qwerty_dist(word, original_word)
      if (temp<mini):
        mini = temp
        corr_word = original_word
    listRow.append(corr_word);
    listRow.append(mini);
    corrected_queries_qwerty[i].append(corr_word)
    if corr_word != word:
      t.add_row(listRow);
print(t);